In [27]:
import sklearn
import xgboost
import pandas as pd
import scipy
import numpy as np
import _pickle as cPickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    auc,
    accuracy_score,
    confusion_matrix,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split,
)
import xgboost as xgb
from scipy.stats import uniform, randint


In [28]:
with open(r"../Data/players.pickle", "rb") as inputfile:
    data = cPickle.load(inputfile)

with open(r"../Data/team.pickle", "rb") as inputfile:
    team = cPickle.load(inputfile)

In [29]:
team

,short_name,code,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,ARS,3,1200,1270,1160,1230,1160,1240
1,AVL,7,1090,1100,1110,1110,1070,1090
2,BOU,91,1050,1070,1070,1110,1040,1070
3,BRE,94,1100,1150,1100,1150,1140,1170
4,BHA,36,1140,1140,1170,1160,1150,1170
5,CHE,8,1210,1230,1230,1220,1200,1210
6,CRY,31,1090,1150,1090,1180,1080,1160
7,EVE,11,1080,1100,1090,1090,1090,1110
8,FUL,54,1070,1090,1090,1090,1100,1120
9,LEI,13,1100,1100,1070,1080,1150,1100


In [30]:
df = data[
    [
        "element_type",
        "status",
        "team",
        "team_code",
        "total_points",
        "web_name",
        "influence",
        "creativity",
        "threat",
        "ict_index",
        "expected_goals",
        "expected_assists",
        "expected_goal_involvements",
        "expected_goals_conceded",
        "influence_rank",
        "influence_rank_type",
        "creativity_rank",
        "creativity_rank_type",
        "threat_rank",
        "threat_rank_type",
        "ict_index_rank",
        "ict_index_rank_type",
        "corners_and_indirect_freekicks_order",
        "direct_freekicks_order",
        "penalties_order",
        "expected_goals_per_90",
        "saves_per_90",
        "expected_assists_per_90",
        "expected_goal_involvements_per_90",
        "expected_goals_conceded_per_90",
        "clean_sheets_per_90",
    ]
]


In [31]:
df = df.merge(team, left_on='team_code', right_on='code').drop(['short_name', 'code', 'team_code', 'team', 'element_type'], axis = 1).set_index('web_name')

In [32]:
df

,status,total_points,influence,creativity,threat,ict_index,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,...,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,clean_sheets_per_90,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
web_name,,,,,,,,,,,,,,,,,,,,,
Cédric,available,2,2.4,1.6,0.0,0.4,0.0000,0.00000,0.00000,0.0000,...,0.00000,0.00000,0.00000,0.00000,1200,1270,1160,1230,1160,1240
Xhaka,available,62,291.8,278.5,228.0,79.8,0.0000,0.00226,0.00226,0.0000,...,0.00017,0.00017,0.00000,0.46312,1200,1270,1160,1230,1160,1240
Elneny,available,4,4.2,5.3,0.0,1.0,0.0000,0.00000,0.00000,0.0000,...,0.00000,0.00000,0.00000,0.00000,1200,1270,1160,1230,1160,1240
Holding,available,4,2.4,0.1,0.0,0.2,0.0000,0.00000,0.00000,0.0000,...,0.00000,0.00000,0.00000,0.00000,1200,1270,1160,1230,1160,1240
Partey,available,42,221.0,141.3,111.0,47.4,0.0000,0.02290,0.02290,0.4928,...,0.00215,0.00215,0.04625,0.65693,1200,1270,1160,1230,1160,1240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diego Costa,available,7,9.2,30.2,294.0,33.0,0.0000,0.00000,0.00000,0.0000,...,0.00000,0.00000,0.00000,0.21845,1100,1100,1130,1130,1080,1080
Traoré,available,12,48.0,16.6,7.0,7.2,0.0329,0.01457,0.04747,1.8082,...,0.00345,0.01124,0.42826,0.00000,1100,1100,1130,1130,1080,1080
Fraser,available,0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.00000,0.0000,...,0.00000,0.00000,0.00000,0.00000,1100,1100,1130,1130,1080,1080


In [33]:
numeric_features = df.T[6:].T.columns.to_list()
numeric_transformer = StandardScaler()

categorical_features = ['status']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

X = df[numeric_features+categorical_features]
y = df.total_points

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', xgb.XGBRegressor())
    ]
)
X = df[numeric_features+categorical_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [34]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'model', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__verbose_feature_names_out', 'preprocessor__num', 'preprocessor__cat', 'preprocessor__num__copy', 'preprocessor__num__with_mean', 'preprocessor__num__with_std', 'preprocessor__cat__categories', 'preprocessor__cat__drop', 'preprocessor__cat__dtype', 'preprocessor__cat__handle_unknown', 'preprocessor__cat__max_categories', 'preprocessor__cat__min_frequency', 'preprocessor__cat__sparse', 'model__objective', 'model__base_score', 'model__booster', 'model__callbacks', 'model__colsample_bylevel', 'model__colsample_bynode', 'model__colsample_bytree', 'model__early_stopping_rounds', 'model__enable_categorical', 'model__eval_metric', 'model__feature_types', 'model__gamma', 'model__gpu_id', 'model__grow_policy', 'model__importance_type', 'model

In [35]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")



def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))



In [36]:
params = {
    'model__colsample_bytree' : uniform(0.7, 0.3),
    'model__gamma' : uniform(0, 0.5),
    'model__learning_rate' : uniform(0.03, 0.3),
    'model__max_depth' : randint(2, 6),
    'model__n_estimators' : randint(100, 150),
    'model__subsample' : uniform(0.6, 0.4)
}


search = RandomizedSearchCV(model, param_distributions=params, random_state=3791, n_iter=200, cv=3, verbose=1, return_train_score=True)

search.fit(X_train, y_train)

report_best_scores(search.cv_results_)


Fitting 3 folds for each of 200 candidates, totalling 600 fits
Model with rank: 1
Mean validation score: 0.938 (std: 0.010)
Parameters: {'model__colsample_bytree': 0.75209784555372, 'model__gamma': 0.4062353063763231, 'model__learning_rate': 0.1553798701456173, 'model__max_depth': 3, 'model__n_estimators': 108, 'model__subsample': 0.8520315590854253}

Model with rank: 2
Mean validation score: 0.938 (std: 0.009)
Parameters: {'model__colsample_bytree': 0.8611557105367702, 'model__gamma': 0.02195226713033155, 'model__learning_rate': 0.28284015840674825, 'model__max_depth': 3, 'model__n_estimators': 135, 'model__subsample': 0.9760455260905787}

Model with rank: 3
Mean validation score: 0.938 (std: 0.007)
Parameters: {'model__colsample_bytree': 0.9318183947107743, 'model__gamma': 0.47666660934161026, 'model__learning_rate': 0.1424244935596373, 'model__max_depth': 3, 'model__n_estimators': 121, 'model__subsample': 0.9288363330193954}



In [42]:
y_pred = search.best_estimator_.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mse


18.32829189017237

In [43]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

4.281155438683857

In [39]:
r2 = r2_score(y_test, y_pred)
r2

0.9637930789174394

In [40]:
check = pd.DataFrame([y_pred, y_test], columns=y_test.index, index=['predict', 'test']).T

In [41]:
check.sort_values(by='test', ascending=False)

,predict,test
web_name,,
Almirón,78.835579,93.0
Martinelli,76.610489,77.0
Foden,68.163406,76.0
Gabriel,67.448433,72.0
White,62.024551,66.0
...,...,...
Parrott,0.158005,0.0
Benteke,0.107811,0.0
Chambers,0.221388,0.0
